In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, BaggingRegressor, AdaBoostClassifier, AdaBoostRegressor, ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.metrics import roc_curve, auc

from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, Nadam
from keras.models import load_model

from data import Data

from neural_network import create_nn

from neural_network_utils import auc

from ml_utils import print_roc, evaluate, best_threshold

from features import FeatureExtractor

import pandas as pd

import numpy as np

In [2]:
embeddings_path = '/Users/nathanhartmann/Desktop/cwi_utils/glove.100d.bin'
#embeddings_path = './glove.100d.bin'
neural_network_model_path = '/Users/nathanhartmann/Desktop/cwi_utils/model.h5'
#neural_network_model_path = './model_2relu_sigmoid.h5'
training_data = ['datasets/english/News_Train.tsv',
                 'datasets/english/WikiNews_Train.tsv',
                 'datasets/english/Wikipedia_Train.tsv',
                ]
dev_data = ['datasets/english/News_Dev.tsv',
            'datasets/english/WikiNews_Dev.tsv',
            'datasets/english/Wikipedia_Dev.tsv'
           ]

data_train = Data(training_data)
data_dev = Data(dev_data)

-----
# Features extraction

In [4]:
fe = FeatureExtractor(embedding_model_path=embeddings_path)
x_train = fe.predict_average_embeddings(data_train.instances)
x_dev = fe.predict_average_embeddings(data_dev.instances)
y_train = data_train.y
y_dev = data_dev.y

-----
# Training neural network

In [82]:
model = create_nn()
#optimizer = Adam(lr=1e-6)
optimizer = Nadam(lr=2e-3, beta_1=0.9, beta_2=0.999, schedule_decay=0.004)
callbacks_list = [ModelCheckpoint('/Users/nathanhartmann/Desktop/model.h5', monitor='val_auc', verbose=1, save_best_only=True, mode='max'),]
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=[auc])

model.fit(
    x_train,
    y_train,
    validation_data=(x_dev, y_dev),
    epochs=20,
    batch_size=32,
    callbacks=callbacks_list)

Train on 27299 samples, validate on 3328 samples
Epoch 1/20
27299/27299 [==============================] - 54s 2ms/step - loss: 0.4925 - auc: 0.8337 - val_loss: 0.5210 - val_auc: 0.8474

Epoch 00001: val_auc improved from -inf to 0.84742, saving model to /Users/nathanhartmann/Desktop/model.h5
Epoch 2/20
27299/27299 [==============================] - 20s 737us/step - loss: 0.4290 - auc: nan - val_loss: 0.4459 - val_auc: 0.8681

Epoch 00002: val_auc improved from 0.84742 to 0.86810, saving model to /Users/nathanhartmann/Desktop/model.h5
Epoch 3/20
27299/27299 [==============================] - 20s 743us/step - loss: 0.3973 - auc: 0.8984 - val_loss: 0.4482 - val_auc: 0.8725

Epoch 00003: val_auc improved from 0.86810 to 0.87246, saving model to /Users/nathanhartmann/Desktop/model.h5
Epoch 4/20
27299/27299 [==============================] - 20s 735us/step - loss: 0.3733 - auc: 0.9100 - val_loss: 0.4252 - val_auc: 0.8844

Epoch 00004: val_auc improved from 0.87246 to 0.88444, saving model t

-----
# Predicting

In [ ]:
model = load_model(neural_network_model_path, custom_objects={'auc':auc})

In [ ]:
pred_train= model.predict(x_train)
pred_dev = model.predict(x_dev)
evaluate(pred_train, pred_dev, y_train, y_dev, optimize='f1', label='Average Embedding Neural Network')